### Having to re-check metrics because I recalculated population ratio using ACS data instead of random json I pulled off the internet. Which caused me to have to re-do the naive predictions. So yeah.

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

### <font color=blue> Some data statistics.

In [ ]:
from epsampling.utils import load_latest_csv

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

_df,name = load_latest_csv('merged_covs_deaths',f'{data_dir}processed/')
_df.drop('Postal',axis=1,inplace=True)
_df.rename({'COVIDhubEns_state_deaths':'State_deaths_ens','True_county_deaths':'Deaths_true',
           'Naive_county_deaths':'Deaths_naive'},axis=1,inplace=True)

print(name)
_df
# display(_df)
# print('\n','Num samples:',len(_df),'\n',
#       'Num dates:',_df.Date.nunique(),'\n',
#       'Num counties:',_df.Fips.nunique(),'\n')

# acs_covs = df.columns[10:]
# death_covs = df.columns[:10]

# df_acs = df[acs_covs]
# df_deaths = df[death_covs]

In [ ]:
# from datetime import datetime

df_deaths = _df[['State_fips', 'State', 'County', 'Fips', 'Date', 'State_deaths_ens',
       'Pop', 'Pop_ratio', 'Deaths_true', 'Deaths_naive']]
# # df_deaths

# d = datetime.today().strftime('%Y%m%d-%H%M%S')
# df_deaths.to_csv(f'{data_dir}/processed/df_deaths_{d}.csv',index=False)

In [ ]:
from epsampling.utils import load_latest_csv

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

_df,name = load_latest_csv('merged_covs_deaths',f'{data_dir}processed/')



fname = 'merged_covs_deaths_20240920-125607.csv'
_df = pd.read_csv(f'{data_dir}processed/{fname}')

# merged_covs_deaths_20240920-125607.csv
_df.drop('Postal',axis=1,inplace=True)
_df.rename({'COVIDhubEns_state_deaths':'State_deaths_ens','True_county_deaths':'Deaths_true',
           'Naive_county_deaths':'Deaths_naive'},axis=1,inplace=True)

# display(_df)
# print('\n','Num samples:',len(_df),'\n',
#       'Num dates:',_df.Date.nunique(),'\n',
#       'Num counties:',_df.Fips.nunique(),'\n')

# acs_covs = df.columns[10:]
# death_covs = df.columns[:10]

# df_acs = df[acs_covs]
# df_deaths = df[death_covs]

In [ ]:
# from datetime import datetime

# df_deaths = _df[['State_fips', 'State', 'County', 'Fips', 'Date', 'State_deaths_ens',
#        'Pop', 'Pop_ratio', 'Deaths_true', 'Deaths_naive']]
# # df_deaths

# d = datetime.today().strftime('%Y%m%d-%H%M%S')
# df_deaths.to_csv(f'{data_dir}/processed/df_deaths_{d}.csv',index=False)

In [ ]:
from epsampling.utils import load_latest_csv
df_deaths,_ = load_latest_csv('df_deaths',f'{data_dir}/processed/')
df_deaths

In [ ]:
from epsampling.utils import get_performance
y_naive = df_deaths['Deaths_naive'].values
y_test = df_deaths['Deaths_naive'].values

# y_test = df_deaths['Deaths_naive'].values

model_names = ['Deaths_naive'] #,'Deaths_true']
model_preds = [y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict

In [ ]:
df_deaths

### <font color=blue> Do naive metrics vary a lot among counties?

In [ ]:
from epsampling.utils import get_performance
import pprint

county_res_dict = {}

fipss = df_deaths.Fips.unique()

for fips in tqdm(df_deaths.Fips.unique(), total = len(fipss)):
    
    df_fips = df_deaths[df_deaths.Fips==fips]
    
    y_naive = df_fips['Deaths_naive'].values
    y_test = df_fips['Deaths_true'].values

    model_names = ['Deaths_naive'] #,'Deaths_true']
    model_preds = [y_naive]
    
    metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
    county_res_dict[fips] = list(metrics_dict.values())[0]
    
df_res = pd.DataFrame([[county,metric,value] 
                       for county,d in county_res_dict.items() 
                       for metric,value in d.items()], 
                       columns = ['Fips','metric','value'])

df = df_res.pivot(index='Fips',columns='metric',values='value')
df.drop('relMAE',axis=1,inplace=True)

df.reset_index(inplace=True)
df.columns.name = None
df.index.name = None
df

In [ ]:
print(f'\n\n* MAE * \nvariance: {round(df.MAE.var(),3)}\n',df.MAE.describe(),
      f'\n\n* MSE * \nvariance: {round(df.MSE.var(),3)}\n',df.MSE.describe(),
      f'\n\n* r2 * \nvariance: {round(df.r2.var(),3)}\n',df.r2.describe())

### <font color=blue> Check what's up with the negative r2.

In [ ]:
bad_counties = df[df.r2 < -1.0 ]
# display(len(bad_counties), bad_counties)

dff = df_deaths[df_deaths.Fips==25007]

dff['Deaths_fixed'] = dff['Deaths_naive']
# dff['Deaths_fixed'] = 0
# dff['Deaths_fixed'] = 1

# # # # # # # # # # # # # # # 
y_naive = dff['Deaths_fixed'].values
# # # # # # # # # # # # # # # 

y_test = dff['Deaths_true'].values

model_names = ['Deaths_naive'] #,'Deaths_true']
model_preds = [y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict

In [ ]:
dff

### <font color=blue> Only look at counties with positive r2.

In [ ]:
import io 
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1) 

In [ ]:
df_pop = df_deaths[['Fips','Pop','Pop_ratio']]
df_pop.drop_duplicates(inplace=True)
dff = df.merge(df_pop,on='Fips')
dff

In [ ]:
## Only plot counties w r2 > 0

dff_good = dff[dff.r2 > 0]
dff_bad = dff[dff.r2 < 0]
len(dff), len(dff_good), len(dff_bad)

In [ ]:
sns.histplot(data=dff_good, x='r2',bins=50)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
sns.histplot(data=dff_good, x='MAE')
# sns.histplot()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
sns.histplot(data=dff_good, x='MAE')
# sns.histplot()
plt.xlim(-10,200)
plt.show()

In [ ]:
sns.scatterplot(data=dff_good, x="Pop_ratio", y="MAE")
plt.show()

In [ ]:
sns.scatterplot(data=dff_good, x="Pop_ratio", y="r2")

### <font color=magenta> Conclusion: Better to look at median r2 (50% percentile) than the acutal r2.